
### Done
* Forward pass


### Missing
* preprocessing
* Backwards pass
* Summaries
* Replay buffer
* target network
* Save model
* Breakout 
* Let it train

# DQN on Breakout



In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }
});

<IPython.core.display.Javascript object>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline

In [3]:
import numpy as np
import tensorflow as tf
import gym
import keras 

import utils

Using TensorFlow backend.


In [4]:
## Hyperparameters

batch_size = 32
training_time = int(50e6) # frames

replay_buffer_size = 1e6

learning_rate = 0.00025
gamma = 0.99
epsilon_init = 1.0
epsilon_end = 0.1
annealing_period = 1e6 # frames


In [5]:
## Parameters
env_name = 'Breakout-v0'
env_name = 'CartPole-v0'



In [6]:
## Derived settings
annealer = utils.Annealer(epsilon_init, epsilon_end, annealing_period)

env = gym.make(env_name)
obs = env.reset()
s_size = list(obs.shape)
a_size = env.action_space.n

obsPlaceholder = tf.placeholder(tf.float32, shape=[None]+s_size, name='obsPlaceholder')
# obsPlaceholder = tf.placeholder(tf.float32, shape=[None]+[84, 84, 4], name='obsPlaceholder')
actionPlaceholder = tf.placeholder(tf.int32, shape=[None], name='actionPlaceholder')

actions_onehot = tf.one_hot(actionPlaceholder, a_size, dtype=tf.float32)
rewardPlaceholder = tf.placeholder(tf.float32, shape=[None], name='rewardPlaceholder')
targetQPlaceholder = tf.placeholder(tf.float32, shape=[None], name='targetQPlaceholder')

print(obsPlaceholder.get_shape())
print(actionPlaceholder.get_shape())
print(actions_onehot.get_shape())


[2017-06-13 11:07:29,864] Making new env: CartPole-v0


(?, 4)
(?,)
(?, 2)


In [7]:
from models import DQN
# model = DQN(obsPlaceholder, a_size)
from models import simple_dense
model = simple_dense(obsPlaceholder, a_size, [64, 64])

qvalues = model.output
qvalue = tf.reduce_sum(tf.multiply(qvalues, actions_onehot), axis=1)
print(qvalues.get_shape())
print(qvalue.get_shape())
action = tf.argmax(qvalues)

loss = tf.reduce_mean(tf.square(targetQPlaceholder - qvalue))
train_op = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
        


(?, 2)
(?,)


In [13]:
obs = env.reset()
env.render()

In [16]:
with tf.Session() as sess:
    obs = env.reset()
    sess.run(tf.global_variables_initializer())
    for f in range(training_time):
        epsilon = annealer.linear(f)
        a = sess.run(action, feed_dict={inputPlaceholder : obs}) if np.random.rand(1) > epsilon \
            else np.random.randint(a_size)

        obs, reward, done, _ = env.step(a)
        # TODO: Preprocess obs
        # TODO: wrap in action repeater
        print(reward)

#         train_dict = {}
#         sess.run(train_op, feed_dict=train_dict)
        if done:
            break

env.render(close=True)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
